En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [ ]:
#%pip install memory-profiler
#%pip install emoji
#%pip install pandas
#%pip install memray

In [29]:
import json
from memory_profiler import profile, memory_usage
from typing import List, Tuple
from datetime import datetime
from read_config import  Config
from base import Base
import pandas as pd
import os
from base import Base
import json
from typing import List, Tuple
from datetime import datetime
from base import Base
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import numpy as np
from base import Base
import cProfile
import re

file_path = Base().twiter_file_path
print(os.cpu_count())

12


## 1. Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. 


### Q1 minimizando uso de memoria

##### Evitamos el uso de estructuras intermedias innecesarias, minimizando el uso de memoria. Además, solo almacena datos relevantes al contar, lo que reduce aún más el consumo de memoria. (no se por qué en el notebook q1_memory se demora menos que q1_time pero en los archivos .py es al revés la cosa, lo siento :c)

In [17]:
import json
from typing import List, Tuple
from datetime import datetime
from collections import defaultdict, Counter
from base import Base

file_path = Base().twiter_file_path
#@profile
def q1_memory(file_path) -> List[Tuple[datetime.date, str]]: 
    date_user_count = defaultdict(Counter)  # Almacena conteos de usuarios por fecha

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data = json.loads(line)
            date = data.get('date')
            user = data.get('user')

            if isinstance(user, dict):
                username = user.get('username')
                if date and username:
                    # Convertir la fecha a formato datetime.date
                    date_only = datetime.fromisoformat(date).date() if isinstance(date, str) else None
                    if date_only:
                        date_user_count[date_only][username] += 1  # Contar tweets por usuario en cada fecha

    # Obtener las 10 fechas con más tweets
    top_dates = sorted(date_user_count.items(), key=lambda x: sum(x[1].values()), reverse=True)[:10]

    # Obtener el usuario con más tweets por cada una de las fechas principales
    result = [(date, user_count.most_common(1)[0][0]) for date, user_count in top_dates]

    return result

cProfile.run("q1_memory(file_path)", sort="tottime")

         2147236 function calls in 3.054 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   117407    1.665    0.000    1.665    0.000 decoder.py:343(raw_decode)
        1    0.952    0.952    3.053    3.053 2449925979.py:9(q1_memory)
   117407    0.126    0.000    1.878    0.000 decoder.py:332(decode)
   117407    0.078    0.000    1.978    0.000 __init__.py:299(loads)
   234814    0.060    0.000    0.060    0.000 {method 'match' of 're.Pattern' objects}
   352221    0.026    0.000    0.026    0.000 {method 'get' of 'dict' objects}
    49772    0.026    0.000    0.053    0.000 <frozen codecs>:319(decode)
    49772    0.026    0.000    0.026    0.000 {built-in method _codecs.utf_8_decode}
   352221    0.018    0.000    0.018    0.000 {built-in method builtins.isinstance}
   117407    0.018    0.000    0.018    0.000 {built-in method fromisoformat}
   234814    0.018    0.000    0.018    0.000 {method 'end' of 're.Match' obj

### Q1 minimizando tiempo

#### A continuación tenemos la función q1_time 

#### En esta prueba hicimos uso de pandas que está diseñado para hacer cálculos en lotes de manera muy eficiente, pero incurriendo en mayor uso de recursos

In [13]:
file_path = Base().twiter_file_path

def q1_time(file_path) -> List[Tuple[datetime.date, str]]: #1490 MiB max, 8.3s
    # Leer el archivo JSON directamente en pandas (asumiendo un formato JSON por línea)
    df = pd.read_json(file_path, lines=True)
    df['date'] = pd.to_datetime(df['date']).dt.date
    df['user'] = df['user'].apply(lambda x: x['username'] if isinstance(x, dict) else None)

    # Obtener las 10 fechas con más tweets
    top_dates = df['date'].value_counts().nlargest(10).index

    # Filtrar los tweets en las 10 fechas principales
    df_top_dates = df[df['date'].isin(top_dates)]

    # Obtener el usuario con más tweets por cada una de las fechas principales
    result = df_top_dates.groupby('date')['user'].agg(lambda x: x.value_counts().idxmax()).reset_index()

    # Convertir a lista de tuplas
    return list(result.itertuples(index=False, name=None))

cProfile.run("q1_time(file_path)", sort="tottime")


         730003 function calls (729435 primitive calls) in 6.325 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    3.885    3.885    3.885    3.885 {built-in method pandas._libs.json.ujson_loads}
        1    0.352    0.352    6.325    6.325 <string>:1(<module>)
        1    0.299    0.299    0.377    0.377 {method 'read' of '_io.TextIOWrapper' objects}
       46    0.179    0.004    0.179    0.004 {method 'split' of 'str' objects}
   117408    0.170    0.000    0.170    0.000 {method 'strip' of 'str' objects}
        1    0.136    0.136    0.456    0.456 _json.py:971(_combine_lines)
        8    0.135    0.017    0.136    0.017 datetimes.py:482(_to_datetime_with_unit)
        1    0.134    0.134    0.211    0.211 construction.py:891(_list_of_dict_to_arrays)
        9    0.110    0.012    0.171    0.019 managers.py:2276(_merge_blocks)
        1    0.100    0.100    4.391    4.391 _json.py:1397(_parse)
        1  

#### Sobre la función anterior, se implementó un threading porque porque divide el DataFrame en bloques más pequeños y los procesa en paralelo usando ThreadPoolExecutor, acelerando el procesamiento de los datos

In [12]:

file_path = Base().twiter_file_path

def process_chunk(chunk):
    chunk['date'] = pd.to_datetime(chunk['date']).dt.date
    chunk['user'] = chunk['user'].apply(lambda x: x['username'] if isinstance(x, dict) else None)
    
    # Obtener los 10 usuarios más comunes por fecha
    top_dates = chunk['date'].value_counts().nlargest(10).index
    df_top_dates = chunk[chunk['date'].isin(top_dates)]
    
    return df_top_dates.groupby('date')['user'].agg(lambda x: x.value_counts().idxmax()).reset_index()

def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:#1521 MiB, 6.5s
    # Leer el archivo JSON directamente en pandas (asumiendo un formato JSON por línea)
    df = pd.read_json(file_path, lines=True)

    # Dividir el DataFrame en bloques para el procesamiento en paralelo
    num_workers = 4
    chunks = np.array_split(df, num_workers)

    # Procesar en paralelo usando ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = list(executor.map(process_chunk, chunks))

    # Combinar los resultados
    combined_result = pd.concat(results).groupby('date')['user'].agg(lambda x: x.value_counts().idxmax()).reset_index()

    # Convertir a lista de tuplas
    return list(combined_result.itertuples(index=False, name=None))

cProfile.run("q1_time(file_path)", sort="tottime")


         494611 function calls (494101 primitive calls) in 6.646 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    3.888    3.888    3.888    3.888 {built-in method pandas._libs.json.ujson_loads}
        1    0.392    0.392    6.646    6.646 <string>:1(<module>)
        1    0.285    0.285    0.364    0.364 {method 'read' of '_io.TextIOWrapper' objects}
       42    0.262    0.006    0.262    0.006 {method 'acquire' of '_thread.lock' objects}
       32    0.220    0.007    0.220    0.007 {method 'split' of 'str' objects}
   117408    0.167    0.000    0.167    0.000 {method 'strip' of 'str' objects}
        1    0.146    0.146    0.231    0.231 construction.py:891(_list_of_dict_to_arrays)
        1    0.135    0.135    0.454    0.454 _json.py:971(_combine_lines)
        8    0.135    0.017    0.136    0.017 datetimes.py:482(_to_datetime_with_unit)
        9    0.115    0.013    0.178    0.020 managers.py:2276(_me

## 2. Los top 10 emojis más usados con su respectivo conteo.

### Q2 minimizando uso de memoria

##### Evitamos el uso de estructuras intermedias innecesarias, minimizando el uso de memoria. Además, solo almacena datos relevantes al contar, lo que reduce aún más el consumo de memoria. (De nuevo, en notebook q2_memory no se demora nada a diferencia de en q2_memory.py :c). Las pruebas realizadas en q2_memory.py nos entregaron un uso de memoria de 104 MiB  pero 304 segundos de ejecución. 

In [18]:
from collections import Counter
import emoji
import json
from typing import List, Tuple

def q2_memory(file_path: str) -> List[Tuple[str, int]]: #104.4 MiB max 304.64s
    emoji_count = {}  # Diccionario para contar emojis
    # Procesar el archivo línea por línea
    with open(file_path, 'r', encoding='utf-8') as file:  # Asegúrate de abrir el archivo con la codificación correcta
        for line in file:
            data = json.loads(line)  # Cargar el objeto JSON
            content = data.get('content')  # Obtener contenido
            # Contar emojis en el contenido
            if isinstance(content, str):
                for c in content:
                    if c in emoji.EMOJI_DATA:  # Verificar si el carácter es un emoji
                        if c in emoji_count:
                            emoji_count[c] += 1  # Incrementar el contador si ya existe
                        else:
                            emoji_count[c] = 1  # Inicializar el contador si no existe
    # Obtener los 5 emojis más comunes
    top_emojis = sorted(emoji_count.items(), key=lambda x: x[1], reverse=True)[:5]
    return top_emojis
cProfile.run("q2_memory(file_path)", sort="tottime")

         1509080 function calls in 4.177 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    2.132    2.132    4.177    4.177 2324164707.py:6(q2_memory)
   117407    1.673    0.000    1.673    0.000 decoder.py:343(raw_decode)
   117407    0.119    0.000    1.877    0.000 decoder.py:332(decode)
   117407    0.076    0.000    1.975    0.000 __init__.py:299(loads)
   234814    0.061    0.000    0.061    0.000 {method 'match' of 're.Pattern' objects}
    49772    0.027    0.000    0.054    0.000 <frozen codecs>:319(decode)
    49772    0.026    0.000    0.026    0.000 {built-in method _codecs.utf_8_decode}
   117407    0.016    0.000    0.016    0.000 {method 'startswith' of 'str' objects}
   234814    0.015    0.000    0.015    0.000 {method 'end' of 're.Match' objects}
   234814    0.013    0.000    0.013    0.000 {built-in method builtins.isinstance}
   117407    0.009    0.000    0.009    0.000 {method 'get' of 'di

### Q2 minimizando tiempo

#### A continuación tenemos la función q2_time 

#### En esta prueba hicimos uso de pandas que está diseñado para hacer cálculos en lotes de manera muy eficiente, pero incurriendo en mayor uso de recursos. En las pruebas realizadas en el archivo q2_time.py la siguiente función nos entregó un uso de memoria máxima de 1462 MiB y 148 s de ejecución.

In [23]:
def q2_time(file_path: str) -> List[Tuple[str, int]]: #1462.6 MiB max 148.20 s
    emoji_count = Counter()
    emoji_set = set(emoji.EMOJI_DATA) 
    df = pd.read_json(file_path, lines=True)
    for content in df['content'].dropna():
        if isinstance(content, str):
            emoji_count.update(c for c in content if c in emoji_set)
    # Obtener los 5 emojis más comunes
    top_emojis = emoji_count.most_common(5)

    return top_emojis

cProfile.run("q2_time(file_path)", sort="tottime")


         1352920 function calls (1352556 primitive calls) in 6.308 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    3.369    3.369    3.369    3.369 {built-in method pandas._libs.json.ujson_loads}
   163043    0.572    0.000    0.572    0.000 1553857045.py:7(<genexpr>)
        1    0.337    0.337    6.308    6.308 <string>:1(<module>)
        1    0.271    0.271    0.346    0.346 {method 'read' of '_io.TextIOWrapper' objects}
   117408    0.167    0.000    0.167    0.000 {method 'strip' of 'str' objects}
       18    0.159    0.009    0.159    0.009 {method 'split' of 'str' objects}
        8    0.129    0.016    0.129    0.016 datetimes.py:482(_to_datetime_with_unit)
        1    0.128    0.128    0.203    0.203 construction.py:891(_list_of_dict_to_arrays)
        1    0.124    0.124    0.440    0.440 _json.py:971(_combine_lines)
        9    0.099    0.011    0.158    0.018 managers.py:2276(_merge_blocks)
    

#### Al igual que en Q1, intentamos implementar threading para minimizar el tiempod de procesamiento. Las pruebas hechas en los archivos .py entregan consistencia con lo explicado en el uso de recursos y tiempo, sin embargo las pruebas en notebook me contradicen. Dichas pruebas entregaron un uso máximo de memoria de 1462 MiB y 7.16 s de procesamiento. 

In [22]:
# Función para procesar un bloque de contenido
def process_chunk(content_chunk, emoji_set):
    emoji_count = Counter()
    for content in content_chunk.dropna():
        if isinstance(content, str):
            emoji_count.update(c for c in content if c in emoji_set)
    return emoji_count


def q2_time_3(file_path: str) -> List[Tuple[str, int]]:#1462.6 MiB max 7.16 s
    emoji_set = set(emoji.EMOJI_DATA)
    df = pd.read_json(file_path, lines=True)
    # Seteamos el número de threads
    num_workers = 4

    # Procesar en paralelo usando ThreadPoolExecutor
    chunks = np.array_split(df['content'], num_workers)  # Dividimos el DataFrame en bloques
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = executor.map(process_chunk, chunks, [emoji_set]*num_workers)  # Pasamos los bloques y el set de emojis

    # Combinar los contadores de emojis
    emoji_count = Counter()
    for result in results:
        emoji_count.update(result)

    # Obtener los 5 emojis más comunes
    top_emojis = emoji_count.most_common(5)
    return top_emojis

cProfile.run("q2_time(file_path)", sort="tottime")

         1352920 function calls (1352556 primitive calls) in 6.893 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    3.825    3.825    3.825    3.825 {built-in method pandas._libs.json.ujson_loads}
   163043    0.579    0.000    0.579    0.000 1553857045.py:7(<genexpr>)
        1    0.335    0.335    6.893    6.893 <string>:1(<module>)
        1    0.287    0.287    0.364    0.364 {method 'read' of '_io.TextIOWrapper' objects}
       18    0.185    0.010    0.185    0.010 {method 'split' of 'str' objects}
   117408    0.177    0.000    0.177    0.000 {method 'strip' of 'str' objects}
        1    0.132    0.132    0.207    0.207 construction.py:891(_list_of_dict_to_arrays)
        1    0.131    0.131    0.467    0.467 _json.py:971(_combine_lines)
        8    0.130    0.016    0.130    0.016 datetimes.py:482(_to_datetime_with_unit)
        9    0.121    0.013    0.179    0.020 managers.py:2276(_merge_blocks)
    

# 3. El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos. 

## Probamos el patron regex que vamos a usar de aquí en adelante

In [22]:
import re

# El patrón regex para menciones lo encontré en la documentación de twitter
# Quizás bastaba el patrón @([A-Za-z0-9_]+)\s pero no si twitter dice eso, quien soy yo para cuestionarlo
# https://github.com/twitter/twitter-text/blob/master/js/src/regexp/validMentionOrList.js

mention_pattern = r'[@＠]([a-zA-Z0-9_]{1,20})(/[a-zA-Z][a-zA-Z0-9_-]{0,24})?'

# Compila la expresión regular
compiled_pattern = re.compile(mention_pattern)

# Ejemplo de texto para probar
text = """@taapsee watch full video #farmersprotest #NoFarmersNoFood
What's left to hear from Modi? Lol #FarmersProtest
@ShekharGupta @khanthefatima @MainaBismee #FarmersProtest
Another farmer, Malkeet Singh from Mahilpur, Hoshiarpur has passed away at the Delhi protest site."""

# Encuentra todas las menciones en el texto
matches = compiled_pattern.findall(text)

# Muestra los resultados
for match in matches:
    print(match[0])


taapsee
ShekharGupta
khanthefatima
MainaBismee


### Q3 minimizando uso de memoria

##### Siguiendo la misma lógica, evitamos el uso de estructuras intermedias innecesarias. Las pruebas realizadas en q3_memory.py nos entregaron un uso de memoria de 107.5 MiB y 12.9 segundos de ejecución. 

In [36]:
mention_pattern = re.compile(r'[@＠]([a-zA-Z0-9_]{1,20})(/[a-zA-Z][a-zA-Z0-9_-]{0,24})?')

def extract_usernames(content: str): 
    for match in mention_pattern.finditer(content):
        yield match.group(1)  # Generar usernames uno a uno, sin almacenar en lista

def q3_memory(file_path: str) -> List[Tuple[str, int]] :#107 MiB, 12.9s
    dict_username_count = defaultdict(int)  # Usar un diccionario simple

    with open(file_path, 'r', buffering=8192) as file:
        for line in file:
            try:
                json_object = json.loads(line)  # Cargar un objeto JSON a la vez
                content = json_object.get('content')

                if content:  # Solo procesar si el contenido del tweet no es nulo
                    for username in extract_usernames(content):  # Extraer usernames
                        dict_username_count[username] += 1  # actualizamos el diccionario con el username encontrado

            except json.JSONDecodeError:
                continue  # Ignorar líneas que no sean JSON válidos

    # Obtener los 10 usernames más comunes
    top_usernames = sorted(dict_username_count.items(), key=lambda x: x[1], reverse=True)[:10]
    return top_usernames

print(q3_memory(file_path))

[('narendramodi', 2261), ('Kisanektamorcha', 1836), ('RakeshTikaitBKU', 1641), ('PMOIndia', 1422), ('RahulGandhi', 1125), ('GretaThunberg', 1046), ('RaviSinghKA', 1015), ('rihanna', 972), ('UNHumanRights', 962), ('meenaharris', 925)]


### Q3 minimizando uso de tiempo

#### En esta prueba hicimos uso de pandas que está diseñado para hacer cálculos en lotes de manera muy eficiente, pero incurriendo en mayor uso de recursos. Las pruebas en q3_time.py nos dió un resultado de uso de memoria de 1465 MiB y 10.52 segundos de ejecución

In [32]:
mention_pattern = re.compile(r'[@＠]([a-zA-Z0-9_]{1,20})(/[a-zA-Z][a-zA-Z0-9_-]{0,24})?')
def extract_usernames(content: str) -> List[str]:
    return [match.group(1) for match in mention_pattern.finditer(content)] #almacenamos resultados en una lista. Almacenar en una lista puede reducir el tiempo total de ejecución.


def q3_time(file_path: str) -> List[Tuple[str, int]]: #1465 MiB, 10.52 s
    username_count = Counter()  # Usar Counter para contar usernames
    df = pd.read_json(file_path, lines=True)  # Leer el archivo JSON
    # Procesar la columna 'content' del DataFrame
    for content in df['content'].dropna():  # Omitir NaN directamente
        if isinstance(content, str):  # Verificar si el contenido es una cadena
            # Extraer usernames y actualizar el contador
            username_count.update(extract_usernames(content))

    # Obtener los 10 usernames más comunes
    top_usernames = username_count.most_common(10)
    return top_usernames

print(q3_time(file_path))

[('narendramodi', 2261), ('Kisanektamorcha', 1836), ('RakeshTikaitBKU', 1641), ('PMOIndia', 1422), ('RahulGandhi', 1125), ('GretaThunberg', 1046), ('RaviSinghKA', 1015), ('rihanna', 972), ('UNHumanRights', 962), ('meenaharris', 925)]


#### Sobre la función anterior, agregamos una lógica para procesar en paralelo, al igual que en todas la otras funciones q*_time. Las pruebas en q3_time.py nos entregaron un uso de memoria de 1470 MiB maximo y 6.7 s de ejecución.

In [33]:
mention_pattern = re.compile(r'[@＠]([a-zA-Z0-9_]{1,20})(/[a-zA-Z][a-zA-Z0-9_-]{0,24})?')
#Función sobre la cual se procesaran los chunks 
def process_chunk(chunk: pd.Series) -> Counter:
    username_count = Counter()
    for content in chunk.dropna():
        if isinstance(content, str):
            username_count.update(extract_usernames(content))
    return username_count


def q3_time(file_path: str) -> List[Tuple[str, int]]:#1470 MiB, 6.7s
    df = pd.read_json(file_path, lines=True)
    num_workers = 4
    chunks = np.array_split(df['content'], num_workers)

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        results = list(executor.map(process_chunk, chunks))

    username_count = Counter()
    for result in results:
        username_count.update(result)

    top_usernames = username_count.most_common(10)
    return top_usernames
print(q3_time(file_path))

[('narendramodi', 2261), ('Kisanektamorcha', 1836), ('RakeshTikaitBKU', 1641), ('PMOIndia', 1422), ('RahulGandhi', 1125), ('GretaThunberg', 1046), ('RaviSinghKA', 1015), ('rihanna', 972), ('UNHumanRights', 962), ('meenaharris', 925)]
